In [0]:
%run ../../config/project_config

In [0]:
from pyspark.sql import functions as F
import random

In [0]:
dbutils.widgets.text("num_records", "5", "Total Records to Simulate")
n = int(dbutils.widgets.get("num_records"))

In [0]:
def generate_data(n):
    # Grab an existing record to simulate an update
    existing = spark.table("zillow.gold.stagnant_inventory_report").limit(1).collect()
    
    data = []
    for i in range(n):
        if i == 0 and len(existing) > 0:
            # Scenario: Record exists in Gold, but now has sales (Updates to Active)
            row = (existing[0]['region_name'], existing[0]['county_name'], 
                   existing[0]['state_name'], "2024-01-01", 10) 
        else:
            # Scenario: New region discovered with 0 sales (Inserts as Stagnant)
            row = (f"999{i}", f"Simulated County {i}", "Kentucky", "2017-12-31", 0)
        data.append(row)
    
    cols = ["region_name", "county_name", "state_name", "last_sale_date", "total_recent_sales"]
    return spark.createDataFrame(data, cols)

In [0]:
# 3. Register as Temp View
df_source = generate_data(n)
df_source.createOrReplaceTempView("simulated_stagnant_source")

print(f"✅ Generated {n} records. Ready for MERGE.")
display(df_source)

✅ Generated 5 records. Ready for MERGE.


region_name,county_name,state_name,last_sale_date,total_recent_sales
21003,Allen,Kentucky,2024-01-01,10
9991,Simulated County 1,Kentucky,2017-12-31,0
9992,Simulated County 2,Kentucky,2017-12-31,0
9993,Simulated County 3,Kentucky,2017-12-31,0
9994,Simulated County 4,Kentucky,2017-12-31,0


In [0]:
%sql
-- Final MERGE for Day 7/8 Deliverable
MERGE INTO zillow.gold.stagnant_inventory_report AS target
USING simulated_stagnant_source AS source
ON target.region_name = source.region_name

-- Case 1: If region exists and total_recent_sales > 0, update the sale date
WHEN MATCHED AND source.total_recent_sales > 0 THEN
  UPDATE SET 
    target.last_sale_date = source.last_sale_date,
    target.total_recent_sales = source.total_recent_sales

-- Case 2: If region is new, insert it into the report
WHEN NOT MATCHED THEN
  INSERT (region_name, county_name, state_name, last_sale_date, total_recent_sales)
  VALUES (source.region_name, source.county_name, source.state_name, source.last_sale_date, source.total_recent_sales);

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
31,27,0,4
